In [3]:
from mvodolagin_personal_imports import *

load_dotenv()

False

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
from urllib.parse import urljoin
import scrapy


class MySpider(scrapy.Spider):
    name = "carolhouse_spider"
    start_urls = ["http://www.carolhouse.com"]

    def parse(self, response, **kwargs):
        # Look for 'Contact Us' link and yield a request with a callback
        # Note: You might need to adjust the selector based on the actual site structure
        contact_us_link = response.xpath('//a[contains(text(), "Contact Us")]/@href').get()
        if contact_us_link:
            yield response.follow(contact_us_link, self.parse_contact, meta={"is_contact_page": True})

        # Find item links - adjust the selector to match your target site's structure
        item_links = response.xpath('//a[contains(@href, "item")]/@href').getall()
        for link in item_links:
            yield response.follow(link, self.parse_item)

        # Follow other potential links to pages like 'Contact Us' or item pages.
        # This is a simplistic approach; you may need to refine how you select links to follow.
        for href in response.xpath("//a/@href").getall():
            yield response.follow(href, self.parse)

    def parse_contact(self, response):
        # Process the 'Contact Us' page here
        # For example, extract the email address or form action
        # Remember to check response.meta['is_contact_page'] if needed
        yield {
            "url": response.url,
            "is_contact_page": response.meta.get("is_contact_page", False),
            # Add any other item fields you want to extract from the contact page
        }

    def parse_item(self, response):
        # Extract item details such as description and images
        # Adjust selectors to match the structure of the item pages you're targeting
        yield {
            "url": response.url,
            "description": response.xpath('//div[contains(@class, "description")]/text()').get(),
            "images": response.xpath("//img/@src").getall(),
            # You might need to join image URLs with the base URL
            "images": [urljoin(response.url, img) for img in response.xpath("//img/@src").getall()],
            # Add any other fields you want to extract
        }

In [6]:
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

# Initialize and run the spider
# Note: You might need to adjust get_project_settings() based on your Scrapy project setup or provide custom settings
process = CrawlerProcess(get_project_settings())

process.crawl(MySpider)
process.start()

2024-04-03 03:59:18 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: texttailor)
2024-04-03 03:59:18 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.11.5, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:34:57) [MSC v.1936 64 bit (AMD64)], pyOpenSSL 23.3.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 41.0.7, Platform Windows-10-10.0.18362-SP0
2024-04-03 03:59:18 [scrapy.addons] INFO: Enabled addons:
[]
2024-04-03 03:59:18 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2024-04-03 03:59:18 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.windows_events._WindowsSelectorEventLoop
2024-04-03 03:59:19 [scrapy.extensions.telnet] INFO: Telnet Password: 596e5f641377a7ac
2024-04-03 03:59:19 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.

In [8]:
# from twisted.internet import asyncioreactor
# asyncioreactor.install()

from scrapy import Selector
from scrapy.http import Request
from twisted.internet import defer, reactor
from scrapy.utils.project import get_project_settings
from scrapy.crawler import CrawlerRunner

# Initialize the crawler runner with your project settings
runner = CrawlerRunner(get_project_settings())


In [14]:
@defer.inlineCallbacks
def fetch_page(url):
    # The callback function to process the downloaded response
    def parse(response):
        return response
    
    # Schedule a request and return the response when it's done
    yield runner.crawl(MySpider, start_urls=[url], parse=parse)

# Example usage to fetch and store the response for inspection
response = await fetch_page('https://www.carolhouse.com')


2024-04-03 04:38:42 [scrapy.addons] INFO: Enabled addons:
[]
2024-04-03 04:38:42 [scrapy.extensions.telnet] INFO: Telnet Password: d1275d29de36976f
2024-04-03 04:38:42 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2024-04-03 04:38:42 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'texttailor',
 'CONCURRENT_REQUESTS_PER_DOMAIN': 4,
 'DOWNLOAD_DELAY': 2,
 'FEED_EXPORT_ENCODING': 'utf-8',
 'NEWSPIDER_MODULE': 'texttailor.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'SPIDER_MODULES': ['texttailor.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor',
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
               '(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
2024-04-03 04:38:42 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.http

RuntimeError: Task got bad yield: <Deferred at 0x2bed07eab00>

In [16]:
fetch_page('https://www.carolhouse.com')

2024-04-03 04:39:11 [scrapy.addons] INFO: Enabled addons:
[]
2024-04-03 04:39:11 [scrapy.extensions.telnet] INFO: Telnet Password: 7158c80353e4b42d
2024-04-03 04:39:11 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2024-04-03 04:39:11 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'texttailor',
 'CONCURRENT_REQUESTS_PER_DOMAIN': 4,
 'DOWNLOAD_DELAY': 2,
 'FEED_EXPORT_ENCODING': 'utf-8',
 'NEWSPIDER_MODULE': 'texttailor.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'SPIDER_MODULES': ['texttailor.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor',
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
               '(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
2024-04-03 04:39:11 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.http

In [17]:
spider = MySpider()

In [19]:
qq = spider.parse('https://www.carolhouse.com')

In [20]:
qq.__next__()

AttributeError: 'str' object has no attribute 'xpath'

In [21]:
import requests

In [22]:
url = "https://www.carolhouse.com/living-room/cabinets/room-type.aspx"

response = requests.get(url)

2024-04-03 05:06:14 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.carolhouse.com:443
2024-04-03 05:06:14 [urllib3.connectionpool] DEBUG: https://www.carolhouse.com:443 "GET /living-room/cabinets/room-type.aspx HTTP/1.1" 403 None


In [25]:
from IPython.display import HTML

In [26]:
HTML(response.text)